In [161]:
import json
import logging
import requests
import math

from requests import Request, Session

from requests.auth import HTTPBasicAuth

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

alt.data_transformers.enable("vegafusion")

import ipywidgets as widgets

In [162]:
def get_config():
    logging.getLogger().debug("get config")

    with open('config.json', "r") as f:
        config = json.load(f)

    return config

In [163]:
def get_collibra(config):
    logging.getLogger().debug("get collibra")

    collibra = {}

    collibra["host"] = f"https://{config['collibra_host']}"

    collibra["username"] = config['collibra_username']

    collibra["password"] = config['collibra_password']

    collibra["endpoint"] = f"{collibra['host']}{config['collibra_api_endpoint']}"

    collibra["session"] = Session()

    collibra.get("session").auth = HTTPBasicAuth(collibra.get("username"), collibra.get("password"))

    return collibra

In [164]:
def get_response(method, url, data, limit, session):
    logging.getLogger().debug("send request")

    offset = 0
    
    results = []

    while True:
        request = Request(
            method=method,
            url=f"{url}&offset={offset}&limit={limit}",
            headers = {'accept': 'application/json', 'Content-Type': 'application/json'},
            data = data
        )

        request = session.prepare_request(request)
   
        response = session.send(request)

        if response.status_code != requests.codes.ok: raise Exception(f'Error: {response.text}') 

        if not response.json()['results']: break

        results = results + response.json()['results']

        offset+=limit

    return results

In [165]:
def get_names_per_database(names_per_database, names):
    dbname = f"{names[0]}>{names[1]}"

    if dbname not in names_per_database:
        names_per_database[dbname] = []

    if names[2].upper() not in names_per_database[dbname]:
        names_per_database[dbname].append(names[2].upper())

    if names[3].upper() not in names_per_database[dbname]:
        names_per_database[dbname].append(names[3].upper())

    return names_per_database

In [166]:
def jaccard_similarity(x, y):
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))

  union_cardinality = len(set.union(*[set(x), set(y)]))

  #return round(intersection_cardinality/float(union_cardinality) *100)
  return intersection_cardinality/float(union_cardinality)

In [167]:
def update_matrix(matrix, x, y, v):
    matrix[x][y] = v

In [168]:
def main():
    config = get_config()

    collibra = get_collibra(config)

    response = get_response("GET", f"{collibra.get('endpoint')}/assets?typePublicIds=Table&typeInheritance=true&sortField=NAME&sortOrder=ASC", None, 1000, collibra.get("session"))

    all_table_names = list(map(lambda x: f"{x['domain']['id']}>{x['name']}".split('>'), filter(lambda x: x['name'].count('>')==3 , response))) # only tables complying with the new edge naming convention c>d>s>t

    names_per_database = {}

    _=list(map(lambda x: get_names_per_database(names_per_database, x), all_table_names)) 

    # build empty matrix 
    n = len(names_per_database.keys())
    arr = np.array([None] *n*n, dtype=float)

    matrix = arr.reshape(n, n)

    # update matrix with similarity
    l=list(names_per_database.keys())

    _=[update_matrix(matrix, lk, ck, jaccard_similarity(names_per_database[l[lk]], names_per_database[l[ck]])) for lk, lv in enumerate(l) for ck, cv in enumerate(l)]

    similarity_df = pd.DataFrame(matrix, columns=list(names_per_database.keys()), index=list(names_per_database.keys()))
    
    similarity_df_melted = pd.melt(similarity_df.reset_index(), id_vars=['index'], value_vars=similarity_df.columns)

    return names_per_database, similarity_df, similarity_df_melted

In [169]:
if __name__ == '__main__':
    names_per_database, similarity_df, similarity_df_melted = main() 

In [170]:
# done

## Select 

Select the database you want to check

In [27]:
database_A = widgets.Select(options=sorted([""]+list(names_per_database.keys())), description='Database', layout=widgets.Layout(width='70%'))

display(database_A)

Select(description='Database', layout=Layout(width='70%'), options=('', '018d9eff-0565-7304-ae0b-f3dd09381b1f>…

## Select 

Choose the database to query against

In [28]:
database_B = widgets.Select(options=sorted([""]+list(names_per_database.keys())), description='Database', layout=widgets.Layout(width='70%'))

display(database_B)

Select(description='Database', layout=Layout(width='70%'), options=('', '018d9eff-0565-7304-ae0b-f3dd09381b1f>…

## Select
 
Select the similarity index you want to start query with

In [87]:
number = widgets.FloatSlider(
    value=.6,
    min=0,
    max=1.0,
    step=0.05,
    description='Similarity',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='70%')
)

display(number)

FloatSlider(value=0.6, continuous_update=False, description='Similarity', layout=Layout(width='70%'), max=1.0,…

In [186]:
mask = f'value > {number.value}' 

if database_A.value != "": mask = f"{mask} and index == '{database_A.value}'"

if database_B.value != "": mask = f"{mask} and variable == '{database_B.value}'"

display(mask)

"value > 0.0 and index == '019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bcpsept22poc' and variable == '019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bg-postgresql-db'"

In [187]:
similarity_df_melted.query(mask)

,index,variable,value
543,019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bcpsept22poc,019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bg-postgr...,0.166667


In [188]:

alt.Chart(similarity_df_melted.query(mask)).mark_rect().encode(
    alt.X('variable', axis=alt.Axis(labels=False, labelAngle=0)).title('Finding severity'),
    alt.Y('index', axis=alt.Axis(labels=False, labelAngle=0)).title('Resource type'),
    alt.Color('value', legend=None).scale(scheme="orangered", reverse=False),
    alt.Text('value'), 
    tooltip=["variable","index","value"]
).properties(title='Similarity', width=940)


alt.Chart(...)

In [193]:
if database_A.value != "" and database_B.value != "":
    print('Database', database_A.value, names_per_database[database_A.value])
    print('Database', database_B.value, names_per_database[database_B.value])
    print('Intersection', set.intersection(*[set(names_per_database[database_A.value]), set(names_per_database[database_B.value])]))
    print('Union', set.union(*[set(names_per_database[database_A.value]), set(names_per_database[database_B.value])]))

Database 019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bcpsept22poc ['POSTGRES', 'PENDING']
Database 019489e5-ba75-7ad9-9ebd-58e7d22e6b65>bg-postgresql-db ['CA_DDS_CMF', 'PENDING', 'CFPB', 'DOH_AHMD', 'DRR', 'MOCK', 'POSTGRES', 'SAMPLE_DATA', 'SAMPLE', 'SEC', 'SRP', 'UTILITY']
Intersection {'PENDING', 'POSTGRES'}
Union {'SAMPLE', 'SAMPLE_DATA', 'SRP', 'PENDING', 'POSTGRES', 'SEC', 'CA_DDS_CMF', 'MOCK', 'DRR', 'DOH_AHMD', 'UTILITY', 'CFPB'}


In [ ]:
#done

In [52]:
# sns.set_theme(style='whitegrid')

# fig=plt.figure(figsize=(16,16))

# mask = np.where(matrix <= 0.8, True, False)

# sns.heatmap(similarity_df, cmap='bwr', xticklabels=False, yticklabels=False, square=True, mask =mask)

# sns.despine(left=True,bottom=True)

# plt.title('Correlation matrix')

# plt.show()

In [ ]:
#done